# SpectraMind V50 — Quickstart

Environment sanity, CLI probes, and a reproducible run hash.

> This notebook calls the **Typer CLI** under the hood. For a full pipeline, prefer:
>
> ```bash
> make e2e
> ```

In [ ]:
import sys, platform, json, os, subprocess, shlex, hashlib
from pathlib import Path

ROOT = Path.cwd()
print("python:", sys.version.replace("\n"," "))
print("platform:", platform.platform())
print("root:", ROOT)

# Ensure expected dirs exist
for p in (ROOT/"outputs", ROOT/"logs", ROOT/"outputs"/"diagnostics"):
    p.mkdir(parents=True, exist_ok=True)
print("prepared:", "outputs/", "logs/", "outputs/diagnostics/")

## Resolve CLI
We try these in order:
1. `poetry run spectramind`
2. `spectramind`
3. `python -m spectramind`

In [ ]:
def choose_cli() -> str:
    def ok(cmd: str) -> bool:
        try:
            subprocess.run(cmd + " --help", shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            return True
        except Exception:
            return False
    for cmd in ("poetry run spectramind", "spectramind", f"{shlex.quote(sys.executable)} -m spectramind"):
        if ok(cmd):
            return cmd
    return "spectramind"  # last resort; may fail visibly

CLI = choose_cli()
CLI

## Self‑test (fast)
Runs deterministic smoke checks (env, imports, CLI help, dry‑runs).

In [ ]:
print("Running fast selftest via CLI…\n")
subprocess.run(CLI + " selftest --fast", shell=True, check=False)

## Config Hash
Compute a sha256 of the active Hydra config to aid reproducibility.

In [ ]:
cfg = Path("configs/config_v50.yaml")
if cfg.exists():
    h = hashlib.sha256(cfg.read_bytes()).hexdigest()
    print("config:", cfg)
    print("sha256:", h)
else:
    print("Config not found:", cfg)